<h4>Writing like Shakespeare</h4>

In [1]:
import io
import sys

import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import pad_sequences

2024-04-07 17:49:40.810731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 17:49:40.810817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 17:49:40.812860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 17:49:40.830836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 17:49:42.205958: W tensorflow/compiler/tf2

In [6]:
def build_data(text, Tx=40, stride=3):
    # input data will be the tokens from the real poems
    X = []
    # labels will be the next character that should come
    Y = []
    for i in range(0, len(text)-Tx, stride):
        X.append(text[i : i + Tx])
        Y.append(text[i + Tx])
    return X, Y


def on_epoch_end(epoch, logs):
    None

def vectorization(X, Y, n_x, char_indices, Tx=40):
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=bool)
    y = np.zeros((m, n_x), dtype=bool)

    for i, sentence in enumerate(X):
        for j, char in enumerate(sentence):
            x[i, j, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1

    return x, y

def sample(preds, chars, temperature):
    preds = np.asarray(preds).astype('float64')

    preds = np.log(preds)/temperature
    expected_preds = np.exp(preds)
    preds = expected_preds / np.sum(expected_preds)

    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p=probas.ravel())

    return out



def generate_output(model, chars, indices_char, char_indices, Tx):
    generated = ''
    user_input = input("Input the first word of the poem you want me to write: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(user_input).lower()
    generated += user_input

    sys.stdout.write("\nPoem:\n")
    sys.stdout.write(user_input)
    for i in range(400):
        x_pred = np.zeros((1, Tx, len(chars)))
        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, chars, temperature=1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue


In [3]:
print('Loading text data...')
text = open("shakespeare.txt", "r").read().lower()
Tx = 40
chars = sorted(list(set(text)))
print(text)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print("Creating training set...")
X, Y = build_data(text, Tx, stride=3)

print('Vectorizing training set...')
x, y = vectorization(X, Y, n_x=len(chars), char_indices=char_indices)
model = load_model('models/model_shakespeare_kiank_350_epoch.h5')

Loading text data...
the sonnets

by william shakespeare

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou contracted to thine own bright eyes,
feed'st thy light's flame with self-substantial fuel,
making a famine where abundance lies,
thy self thy foe, to thy sweet self too cruel:
thou that art now the world's fresh ornament,
and only herald to the gaudy spring,
within thine own bud buriest thy content,
and tender churl mak'st waste in niggarding:
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.

when forty winters shall besiege thy brow,
and dig deep trenches in thy beauty's field,
thy youth's proud livery so gazed on now,
will be a tattered weed of small worth held:  
then being asked, where all thy beauty lies,
where all the treasure of thy lusty days;
to say within thine own deep sunken eyes,
were an all-eating shame,

2024-04-07 17:49:45.337079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 17:49:45.601402: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 17:49:45.601501: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 17:49:45.606446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 17:49:45.606652: I external/local_xla/xla/stream_executor

In [4]:
# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=256, epochs=1)

123/123 [==============================] - 113s 873ms/step - loss: 2.8612


In [7]:
generate_output(model, chars, indices_char, char_indices, Tx)


Poem:
Thoud of hame's labgel,
aven is the light and fer the placceed fill,
which forse of goadwst woy to wo swoat asheece,
yeu if thy seefury doth my ge, all mine,
but falled this wino more,
when hafth the letsur med oun belo, this wosl,
pe'turr ast in your dralde,
and mine own know weath with delend hum chect,
in not of had my oft i as aatent to beors.
nath heaven this camence con to with mind,
that to lin